https://github.com/skchandrappa/dsa-cookbook.git


In [5]:
! pip install python-dotenv

In [11]:
class Website:
  url : str
  title: str
  text : str

  def __init__(self, url: str):
    self.url = url

    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    self.title = soup.title.string if soup.title else None

    if soup.body:
      for irrelevant in soup.body(["script", "style","img","input"]):
        irrelevant.decompose()
    else:
      self.text = ""
    links = [link.get('href') for link in soup.find_all('a')]
    self.links = [link for link in links if link ] #and link.startswith('/')

  def get_contents(self):
    return f"Webpage title: \n{self.title}\n Webpage contents:\n{self.text}\n\n"

    # self.text = soup.body.get_text(separator="\n", strip=True)

In [14]:
import os
import requests
# from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI
import time


# load_dotenv()
openai = OpenAI(api_key=api_key)

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "The contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

def summarize(url):
    website = Website(url)
    try:
      response = openai.chat.completions.create(
          model = "gpt-4o-mini",
          messages = messages_for(website)
      )
      return response.choices[0].message.content
    except openai.error.RateLimitError:
      print("Rate limit exceeded. Waiting for 60 seconds...")
      time.sleep(60)  # Wait for 60 seconds before retrying
      return summarize(url)

summarize("https://www.ndtv.com/")


AttributeError: 'Website' object has no attribute 'text'

In [16]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 102.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 9.0 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.24.7
    Uninstalling huggingface-hub-0.24.7:
      Successfully uninstalled huggingface-hub-0.24.7


In [17]:
import gradio as gr

In [18]:
def shout(text):
    print(f"Shout has been called with input {text}")
    return text.upper()

In [20]:
shout("hey hey hey")

Shout has been called with input hey hey hey


'HEY HEY HEY'

In [21]:
gr.Interface(
      fn=shout,
      inputs="textbox",
      outputs="textbox"
      ).launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c44f84bb313f64a54e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [3]:
!pip install google-search-results --upgrade

In [5]:
import serpapi
import json
from serpapi import GoogleSearch # Add this import statement

params = {
  "q": "Coffee",
  "location": "Austin, Texas, United States",
  "hl": "en",
  "gl": "us",
  "google_domain": "google.com",
  "api_key": "key"
}

search = GoogleSearch(params) # Now GoogleSearch class is available
results = search.get_dict()
print(json.dumps(results))

{"search_metadata": {"id": "6733aafffab2738ed322718d", "status": "Success", "json_endpoint": "https://serpapi.com/searches/db09ace283ab432e/6733aafffab2738ed322718d.json", "created_at": "2024-11-12 19:22:39 UTC", "processed_at": "2024-11-12 19:22:39 UTC", "google_url": "https://www.google.com/search?q=Coffee&oq=Coffee&uule=w+CAIQICIaQXVzdGluLFRleGFzLFVuaXRlZCBTdGF0ZXM&hl=en&gl=us&sourceid=chrome&ie=UTF-8", "raw_html_file": "https://serpapi.com/searches/db09ace283ab432e/6733aafffab2738ed322718d.html", "total_time_taken": 2.96}, "search_parameters": {"engine": "google", "q": "Coffee", "location_requested": "Austin, Texas, United States", "location_used": "Austin,Texas,United States", "google_domain": "google.com", "hl": "en", "gl": "us", "device": "desktop"}, "search_information": {"query_displayed": "Coffee", "total_results": 3110000000, "time_taken_displayed": 0.42, "organic_results_state": "Results for exact spelling"}, "local_map": {"link": "https://www.google.com/search?sca_esv=14c9

In [8]:
!pip install google-search-results

  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32009 sha256=411dbc37d556c22bdce5919ce3f78d57f18823c04cb0c38c62a345747bd5005c
  Stored in directory: /root/.cache/pip/wheels/d3/b2/c3/03302d12bb44a2cdff3c9371f31b72c0c4e84b8d2285eeac53
Successfully built google-search-results
